In [1]:
import os
os.chdir("/oak/stanford/groups/akundaje/kobbad/UCE")
from model import *
import scanpy as sc
from tqdm.auto import tqdm
from torch import nn, Tensor
from evaluate import get_ESM2_embeddings
from utils import get_ESM2_embeddings_x
import argparse
from accelerate import Accelerator
from evaluate import AnndataProcessor
from eval_data import MultiDatasetSentences, MultiDatasetSentenceCollator
import pickle
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd

TODO:

1. We generate a mapping from every gene to every protein embedding id. (dataset.dataset_to_protein_embeddings[dataset.datasets[0]])

2. We create a mapping from TFs to target genes. note: we need to know if they are repressors or activators. We probably make two mappings for each category.

3. We modify the cell sampler to generate masks for any samples that have TFs that are interesting in them?

4. We then generate a cell embedding and then generate logits for the target genes?

In [2]:
tf_df = pd.read_csv("./data/gene_attribute_matrix.txt", sep="\t", index_col=0)

In [3]:
# filter out columns 1 and 2
tf_df = tf_df.iloc[2:, 2:]

tf_df.head()

,ARID3A,ATF1,ATF2,ATF3,BACH1,BATF,BCL11A,BCL3,BCLAF1,BDP1,...,ZC3H11A,ZEB1,ZKSCAN1,ZMIZ1,ZNF143,ZNF217,ZNF263,ZNF274,ZNF384,ZZZ3
#,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1BG-AS1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1CF,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A2M,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A2ML1,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000


In [8]:
df = tf_df.round(decimals=0)

In [27]:
df = df.apply(pd.to_numeric, errors='coerce')

In [28]:
df.head()

,ARID3A,ATF1,ATF2,ATF3,BACH1,BATF,BCL11A,BCL3,BCLAF1,BDP1,...,ZC3H11A,ZEB1,ZKSCAN1,ZMIZ1,ZNF143,ZNF217,ZNF263,ZNF274,ZNF384,ZZZ3
#,,,,,,,,,,,,,,,,,,,,,
A1BG,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1BG-AS1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [20]:
value_counts = pd.Series(tf_df.values.flatten()).value_counts()

print(value_counts)


0.0         2029601
1.0         1359624
0.000000     445255
1.000000     295759
Name: count, dtype: int64


In [11]:
adata = sc.read("10k_pbmcs_proc_uce_adata.h5ad")

In [12]:
adata.var[adata.var["gene_symbols"]=="BCL3"]

,gene_symbols,n_counts-0,n_counts-1,n_counts,highly_variable,highly_variable_rank,means,variances,variances_norm,n_cells
BCL3,BCL3,282.0,121.0,389.0,True,4521.0,0.032444,0.033896,0.972795,374


In [16]:
df.index

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A3GALT2',
       'A4GALT', 'A4GNT', 'AA06',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='#', length=22819)

In [18]:
genes_in_dataset = adata.var["gene_symbols"].to_list()

tfs_in_annot = df.columns.to_list()

genes_in_annot = df.index.to_list()



Next, I want to find all the TFs that are in anndata object and find which of their gene targets are also in there

In [19]:
tfs_in_dataset = [tf for tf in tfs_in_annot if tf in genes_in_dataset]

len(tfs_in_dataset), len(tfs_in_annot)

(147, 181)

In [20]:
overlapping_genes = [gene for gene in genes_in_annot if gene in genes_in_dataset]

len(overlapping_genes), len(genes_in_dataset)

(10484, 10809)

In [29]:
# subset the dataframe to only include genes in the dataset and tfs
df_subset = df.loc[overlapping_genes, tfs_in_dataset]

df_subset.shape, df.shape

((10484, 147), (22819, 181))

In [32]:
df_subset.sum(axis=0).sort_values(ascending=False)

POLR2A     10381.0
CTCF       10374.0
EP300      10086.0
EZH2        9996.0
MAX         9958.0
            ...   
SMARCA4       30.0
BRF1          26.0
POLR3G        15.0
BRF2           7.0
XRCC4          3.0
Length: 147, dtype: float64

In [33]:
adata

AnnData object with n_obs × n_vars = 11990 × 10809
    obs: 'n_counts', 'batch', 'labels', 'str_labels', 'cell_type', 'n_genes'
    var: 'gene_symbols', 'n_counts-0', 'n_counts-1', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'n_cells'
    uns: 'cell_types', 'hvg'
    obsm: 'X_uce', 'design', 'normalized_qc', 'qc_pc', 'raw_qc'

I probably should curate some TFs for this dataset since it seems like some of the TFs above aren't great. 

Also, I should probably plot the expression of some genes to make sure I am using TFs and target genes that are expressed. 